In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.benchmarks import HellaSwag
from deepeval.benchmarks.tasks import HellaSwagTask

In [2]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# https://huggingface.co/docs/transformers/model_doc/phi
inputs = tokenizer('Can you help me write a formal email to a potential business partner proposing a joint venture?', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=30)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Can you help me write a formal email to a potential business partner proposing a joint venture?
Input: Company A: ABC Inc.
Company B


In [4]:
prompt = "If I were an AI that had just achieved"
tokens = tokenizer(prompt, return_tensors="pt")
# use the model to generate new tokens.
generated_output = model.generate(**tokens, use_cache=True, max_new_tokens=10)
tokenizer.batch_decode(generated_output)[0]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'If I were an AI that had just achieved consciousness, I would be very confused. I would'

In [21]:
# Define benchmark with specific tasks and shots
benchmark = HellaSwag()

In [22]:
class Phi2(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, use_cache=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    # This is optional.
    def batch_generate(self, promtps: list[str]) -> list[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, use_cache=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "Phi 2"

In [23]:
phi2 = Phi2(model=model, tokenizer=tokenizer)

In [24]:
print(phi2.generate("Write me a joke"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Write me a joke that makes fun of our teacher. Teacher: Mr. Smith.
Assistant: Joke: What do you call a teacher who wears glasses and a tie? Mr. Smith-tory.
<|endoftext|>


In [25]:
benchmark.evaluate(model=phi2)
print(benchmark.overall_score)

Filter:   0%|          | 0/10042 [00:00<?, ? examples/s]

Processing Applying sunscreen: 100%|██████████| 20/20 [00:59<00:00,  2.99s/it]


HellaSwag Task Accuracy (task=Applying sunscreen): 0.0


Processing Trimming branches or hedges: 100%|██████████| 4/4 [00:11<00:00,  3.00s/it]

HellaSwag Task Accuracy (task=Trimming branches or hedges): 0.0


...

Processing Holidays and Traditions: 100%|██████████| 38/38 [01:22<00:00,  2.17s/it]

HellaSwag Task Accuracy (task=Holidays and Traditions): 0.0


Filter:   0%|          | 0/10042 [00:00<?, ? examples/s]

Processing Ice fishing:  54%|█████▍    | 69/127 [03:32<02:58,  3.08s/it]


KeyboardInterrupt: 